In [15]:
# Install missing packages
%pip install pandas
%pip install numpy
%pip install nltk
%pip install scikit-learn
# --- IGNORE ---


# Importing libraries
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import string
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ANGEL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# Download and load the dataset
url = 'https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv'
df = pd.read_csv(url, sep='	', header=None, names=['label', 'message'])
# Display the first 5 rows
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
# Save the loaded dataset to a CSV file
# This will create 'sms_spam_dataset.csv' in the current working directory
df.to_csv('sms_spam_dataset.csv', index=False)
print('Dataset saved as sms_spam_dataset.csv')

Dataset saved as sms_spam_dataset.csv


In [18]:
# Count of spam and ham messages
df['label'].value_counts()

label
ham     4825
spam     747
Name: count, dtype: int64

In [19]:
# Calculate percentage of each class
df['label'].value_counts(normalize=True) * 100

label
ham     86.593683
spam    13.406317
Name: proportion, dtype: float64

In [20]:
# Define stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and numbers, but keep spaces
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenize
    tokens = text.split()
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back to string
    return ' '.join(tokens)

# Apply preprocessing
df['clean_message'] = df['message'].apply(preprocess_text)
# Show the first 5 cleaned messages
df[['message', 'clean_message']].head()

,message,clean_message
0,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts st ...
3,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,"Nah I don't think he goes to usf, he lives aro...",nah dont think goes usf lives around though


In [21]:
# Upsample the minority class (spam) to balance the dataset
from sklearn.utils import resample
# Separate majority and minority classes
df_ham = df[df['label'] == 'ham']
df_spam = df[df['label'] == 'spam']
# Upsample spam
df_spam_upsampled = resample(df_spam, 
                             replace=True,     # sample with replacement
                             n_samples=len(df_ham),    # match number in majority class
                             random_state=42) # reproducible results
# Combine back to a single DataFrame
df_balanced = pd.concat([df_ham, df_spam_upsampled])
# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
print('Balanced label counts:')
print(df_balanced['label'].value_counts())
# Use df_balanced for the next steps (feature extraction, etc.)
df = df_balanced

Balanced label counts:
label
ham     4825
spam    4825
Name: count, dtype: int64


In [22]:
# Create TF-IDF features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['clean_message'])
y = df['label'].map({'ham': 0, 'spam': 1})  # 0 for ham, 1 for spam

In [23]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Training samples:', X_train.shape[0])
print('Testing samples:', X_test.shape[0])

Training samples: 7720
Testing samples: 1930


In [24]:
# Build the model
model = MultinomialNB()

In [25]:
# Train the model
model.fit(X_train, y_train)

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [26]:
# Predict on test data
y_pred = model.predict(X_test)

# Calculate metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f'Accuracy: {acc:.2f}')
print(f'Precision: {prec:.2f}')
print(f'Recall: {rec:.2f}')
print(f'F1-Score: {f1:.2f}')
print('Confusion Matrix:')
print(cm)

# Detailed classification report
print('Classification Report:')
print(classification_report(y_test, y_pred, target_names=['Ham', 'Spam']))

Accuracy: 0.98
Precision: 0.97
Recall: 0.99
F1-Score: 0.98
Confusion Matrix:
[[942  33]
 [  9 946]]
Classification Report:
              precision    recall  f1-score   support

         Ham       0.99      0.97      0.98       975
        Spam       0.97      0.99      0.98       955

    accuracy                           0.98      1930
   macro avg       0.98      0.98      0.98      1930
weighted avg       0.98      0.98      0.98      1930



In [27]:
# Function to predict custom input
def predict_message(msg):
    clean = preprocess_text(msg)
    vect = vectorizer.transform([clean])
    pred = model.predict(vect)[0]
    return 'Spam' if pred == 1 else 'Ham'

# User input
custom_msg = input('Enter an email message to classify: ')
print('Prediction:', predict_message(custom_msg))

Prediction: Spam


In [28]:
# Debugging: Check data, model, and predictions

# 1. Check class balance
print('Label counts:')
print(df['label'].value_counts())
print('\nLabel percentages:')
print(df['label'].value_counts(normalize=True) * 100)

# 2. Show a few cleaned messages
print('\nSample cleaned messages:')
print(df[['message', 'clean_message']].head())

# 3. Show classification report
from sklearn.metrics import classification_report
print('\nClassification Report on Test Set:')
print(classification_report(y_test, y_pred, target_names=['Ham', 'Spam']))

# 4. Test with obvious spam and ham
test_samples = [
    'Congratulations! You have won a free lottery. Click here to claim.',
    'URGENT! Your account has been compromised. Send your password now.',
    'Hi, are we still meeting for lunch today?',
    'Reminder: Your appointment is scheduled for tomorrow.'
]
for msg in test_samples:
    print(f'Input: {msg}')
    print('Prediction:', predict_message(msg))
    print('-' * 40)

Label counts:
label
ham     4825
spam    4825
Name: count, dtype: int64

Label percentages:
label
ham     50.0
spam    50.0
Name: proportion, dtype: float64

Sample cleaned messages:
                                             message  \
0            So what about you. What do you remember   
1  My life Means a lot to me, Not because I love ...   
2  E admin building there? I might b slightly ear...   
3               Sorry, went to bed early, nightnight   
4  Latest News! Police station toilet stolen, cop...   

                                       clean_message  
0                                           remember  
1  life means lot love life love people life worl...  
2  e admin building might b slightly earlier ill ...  
3                    sorry went bed early nightnight  
4  latest news police station toilet stolen cops ...  

Classification Report on Test Set:
              precision    recall  f1-score   support

         Ham       0.99      0.97      0.98       975
     